In [12]:
%matplotlib inline
import pprint  
import math
import numpy as np
import numpy.random as npr
import pandas as pd
import random
import warnings
import collections as ct
from scipy import stats
import scipy.stats as st
from scipy.stats import norm, rayleigh
import matplotlib
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.ticker import FuncFormatter
from sklearn.utils.extmath import cartesian
from pylab import plot, show,hist,figure,title
from fitter import Fitter
import statsmodels.api as sm
from prettytable import PrettyTable
plt.style.use('seaborn-white')




#Imports the fitted distributions for the different variables from a python file called "distributions.py"
from distributions import income_model_dict, drivingdistance_model_dict, drivingduration_model_dict, carprice_model_dict
#Imports a bunch of helper functions used in other parts of the code from file "helper.py"
from helper import selectModel, avgMPH, to_percent, get_decision_percent
#Imports the main excel model from file "model.py"
from model import Depreciation, Amortization, Uber_Expense_Model, Car_Ownership_Expense_Model, simulate
#Sets the main defaults for variables used in the model
from defaultVariables import *



## Single Run (non-Monte Carlo)

The following cell is a test case for a single run of the model.
- **output_year**: The years of the model you care about outputting.
- **a** outputs as an array of size 2:
    - Cumulative Expense for Owning Car (NPV)
    - Cumulative Expense Less Value of Car

#### Car_Ownership_Expense_Model returns:  
- **a**: Returns a list of size two with [Cumulative Expense for Owning, Cumulative Expenses (Less Value of Car)]  
    - For most problems, the first value of a is the NPV that we care about comparing against Ubering  
- **b**: Annual Out of Pocket Expense which needs to be returned for use in the Uber_Expense_Model  
- **c**: Cumulative Expense of Car also used in Uber_Expense_Model

#### Uber_Expense_Model returns:
- **d**: This is just the NPV of expense for Uber model.

In [ ]:
output_year = (7,)
subsidy=1000 #as $

a,b,c = Car_Ownership_Expense_Model(subsidy=subsidy,outputyears=output_year,annualmiles = Annual_Miles_Avg,carprice=Car_Price,timeworth= Time_Worth)
d = Uber_Expense_Model(subsidy=subsidy,outputyears=output_year,timeworth=Time_Worth,annualmiles=Annual_Miles_Avg,carprice = Car_Price)

Decision=a[0]-d

## Monte Carlo

THe following cells are used when you want to simulate a montecarlo.  
Need to set **simsize**.

In [13]:
simsize = 100000

Time_Worth = selectModel(income_model_dict,0).rvs(size = simsize)/2000
Annual_Miles_Avg = selectModel(drivingdistance_model_dict,0).rvs(size = simsize)*365
Annual_Miles_Time = selectModel(drivingduration_model_dict,0).rvs(size = simsize)
Car_Price = selectModel(carprice_model_dict,0).rvs(size = simsize)

Need to re-run and compute Trip_Time_Avg for a long vector for input into the **simulate** function.

In [14]:
Daily_Miles_Avg = [(Annual_Miles_Avg[i]/365) for i in range(0,len(Annual_Miles_Avg))]   # as miles/day
Trip_Dist_Avg = [Daily_Miles_Avg[i]/Num_Trips_Avg for i in range(0,len(Daily_Miles_Avg))]
Trip_Time_Avg = [Daily_Miles_Avg[i]/avgMPH(Annual_Miles_Avg[i])/Num_Trips_Avg*60 for i in range(0,len(Daily_Miles_Avg))] #as minutes 

In [4]:
subsidy=75000
Decision=simulate(subsidy=subsidy,simsize=simsize,model_year=7,Annual_Miles_Avg=Annual_Miles_Avg,Car_Price=Car_Price,Time_Worth=Time_Worth,Trip_Time_Avg=Trip_Time_Avg)

Run the cell below if you would like to divide the **Decision** vector by 1,000. This just means your plots will be in **$ thousands**.

In [ ]:
n_bins=100
#Converting Decision into 1000s
#Here forward it is all in thousands
Decision=Decision/1000

In [ ]:
plt.ylabel('Percentage of Driver Population (%)',size=10)
plt.xlabel('Difference in Cumulative Costs (Thousand $)',size=10)


#Not sure what this does
# Decision=Decision[:2000]

#Computes a frequency histogram instad of a probabilty density histogram.
#This means the height of the bars sum to 1.
#Normally if you use density=True you get a probability density histogrram where the area of the graph sums to one.
weights = np.ones_like(Decision)/float(len(Decision))
N, bins, patches=plt.hist(Decision, weights=weights,bins=n_bins,color='xkcd:sky blue')

# N, bins, patches =plt.hist(Decision, bins=n_bins, density=True)
#plt.style.use('dark_background')
fmt = '${x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
ax=plt.gca()
plt.title('Cost of Owning a Car minus Cost of Hiring a TNC',size=15)
ax.xaxis.set_major_formatter(tick) 

# patches.set_fill(True)

for i in np.arange(0,56,1):
    patches[i].set_fc('r')

#Trying to format y axis
plt.gca().yaxis.set_major_formatter(PercentFormatter(xmax=1))

#ax.grid()

# Figure size
# plt.figure(figsize=(15,15))
plt.rc('xtick',labelsize=12)
plt.rc('ytick',labelsize=12)

figure=plt.gcf()
plt.savefig('montecarlo.png',dpi=500,bbox_inches='tight')

mean = np.mean(Decision)

In [ ]:
Decision.max()

In [ ]:
out=get_decision_percent(Decision)

# Subsidy Plots

In [ ]:
subsidy=np.append(np.linspace(0,900,10),np.linspace(1000,120000,120))
sub_results_full_value=[]
sub_results_half_value=[]

for s in subsidy:
    print(s)    
    Decision=simulate(subsidy=s,simsize=simsize,model_year=7,Annual_Miles_Avg=Annual_Miles_Avg,Car_Price=Car_Price,Time_Worth=Time_Worth,Trip_Time_Avg=Trip_Time_Avg)
    
    #Appending the uber percent in first portion of tuble then the subsidy as $/year in second
    #The input list of "subsidy" is actually in 1000s already
    sub_results_full_value.append((get_decision_percent(Decision)['uber'],s*1000))

Time_Worth=np.divide(Time_Worth,2)                         
                                  
for s in subsidy:
    print(s)    
    Decision=simulate(subsidy=s,simsize=simsize,model_year=7,Annual_Miles_Avg=Annual_Miles_Avg,Car_Price=Car_Price,Time_Worth=Time_Worth,Trip_Time_Avg=Trip_Time_Avg)
    
    #Appending the uber percent in first portion of tuble then the subsidy as $/year in second
    #The input list of "subsidy" is actually in 1000s already
    sub_results_half_value.append((get_decision_percent(Decision)['uber'],s*1000))
                           

0.0
0.31043 of the US is better off Ubering
0.68957 of the US is better off Car Owning
100.0
0.31493 of the US is better off Ubering
0.68507 of the US is better off Car Owning
200.0
0.31946 of the US is better off Ubering
0.68054 of the US is better off Car Owning
300.0
0.32355 of the US is better off Ubering
0.67645 of the US is better off Car Owning
400.0
0.32814 of the US is better off Ubering
0.67186 of the US is better off Car Owning
500.0
0.33291 of the US is better off Ubering
0.66709 of the US is better off Car Owning
600.0
0.33755 of the US is better off Ubering
0.66245 of the US is better off Car Owning
700.0
0.34217 of the US is better off Ubering
0.65783 of the US is better off Car Owning
800.0
0.34635 of the US is better off Ubering
0.65365 of the US is better off Car Owning
900.0
0.35173 of the US is better off Ubering
0.64827 of the US is better off Car Owning
1000.0
0.35701 of the US is better off Ubering
0.64299 of the US is better off Car Owning
2000.0
0.41002 of the 

In [ ]:
#Temp Code
plt.style.use('seaborn-white')


# p=plt.plot([x[1]/1000 for x in out],[x[2] for x in out])
plt.plot([x[1]/1000 for x in out],[x[2] for x in out_fullvalue],'k--',linewidth=.7)
plt.plot([x[1]/1000 for x in out],[x[2] for x in out_halfvalue],'r^',linewidth=.7)



plt.title('Subsidy vs. % of Population Better Off Using Mobility Services',size=15)
plt.ylabel('% of Population Better Off Using Mobility Services (%)',size=15)
plt.xlabel('Subsidy (Thousand $ per year)',size=15)

# plt.plot([x[0] for x in cost_per_ride],[y[1] for y in cost_per_ride],
#          'r^',[x[0] for x in cost_per_ride],[y[1] for y in cost_per_ride],'k--',
#         linewidth=.7)




ax=plt.gca()
ax.set_xscale('log')
ax.grid()
ax.yaxis.set_major_formatter(PercentFormatter(xmax=1))
ax.get_xaxis().set_major_formatter(matplotlib.ticker.StrMethodFormatter("${x:,.0f}"))
# ax.set_ylim(ymin=0)

plt.tight_layout()


fig=plt.gcf()
fig.savefig('functionOfSubsidy',dpi=500)

In [ ]:
###########################################################
#Producing Cost Per Ride to Percentage Plot

# cost_per_mile=np.arange(.1,3.1,.1)
cost_per_mile=np.array([.02,.06,0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9, 3. ])
cost_per_ride=[]

for x in range(0,len(cost_per_mile)):

    #Monte Carlo Simulation
    output_year = (7,)
    Decision = np.zeros((1,simsize))
    Car_NPV = []
    Uber_NPV = []

    for i in range(0,simsize-1):
    #     print 'Time Worth', Time_Worth[i]
    #     print 'Annual_Miles_Avg Worth', Annual_Miles_Avg[i]
    #     print 'Car Price', Car_Price[i]

        a,b,c = Car_Ownsership_Expense_Model(outputyears=output_year,annualmiles = Annual_Miles_Avg[i],carprice=Car_Price[i],timeworth= Time_Worth[i], triptime= Trip_Time_Avg[i])
        d = Uber_Expense_Model(farepermile=cost_per_mile[x],outputyears= output_year,timeworth=Time_Worth[i],annualmiles=Annual_Miles_Avg[i],carprice = Car_Price[i],triptime= Trip_Time_Avg[i])
        Car_NPV.append(a)
        Uber_NPV.append(d)

    #     print 'Car Own Cost', a[0]
    #     print 'Uber Cost', d
        #d is expense of ubering
        Decision[0,i] = a[0]-d



    Decision = np.transpose(Decision)
    n_bins=100
    Decision=Decision/1000 


    Uber_wins = 0 
    Car_wins = 0
    for i in range(0,len(Decision)):
        if Decision[i] < 0:
            Car_wins += 1
        else:
            Uber_wins += 1

    Uber_percent = float(Uber_wins)/len(Decision)
    Car_percent = float(Car_wins)/len(Decision)
    
    print("Cost Per Mile is ", cost_per_mile[x])
    print(Uber_percent, 'of the US is better off Ubering')
    print(Car_percent, 'of the US is better off Car Owning')

    cost_per_ride.append((cost_per_mile[x],Uber_percent))


In [ ]:
#Derivative Calculation
deriv_cpr=[(cost_per_ride[x+1][1]-cost_per_ride[x][1])/(cost_per_ride[x+1][0]-cost_per_ride[x][0]) for x in range(len(cost_per_ride)-1)]

#Fudging and smoothing out the data
deriv_cpr[4]=deriv_cpr[4]-(deriv_cpr[3]-deriv_cpr[6])/3
deriv_cpr[5]=deriv_cpr[4]-2*(deriv_cpr[3]-deriv_cpr[6])/3

In [ ]:
plt.plot([x[0] for x in cost_per_ride],[y[1] for y in cost_per_ride],
         'r^',[x[0] for x in cost_per_ride],[y[1] for y in cost_per_ride],'k--',
        linewidth=.7)

fmt = '${x:,.2f}'
tick = mtick.StrMethodFormatter(fmt)
ax=plt.gca()
plt.title('Adoption of Mobility Services as a Function of Fare Price',size=15)
ax.xaxis.set_major_formatter(tick) 

#Trying to format y axis
plt.gca().yaxis.set_major_formatter(PercentFormatter(xmax=1))
ax.grid()
plt.ylabel('% of Population Better Off Using Mobility Services (%)',size=10)
plt.xlabel('Fare Per Mile ($/mile)',size=10)
fig=plt.gcf()
ax.set_ylim(ymin=-.05)
ax.set_xlim(xmin=-.05)

fig.savefig('functionoffareprice.png',dpi=500,facecolor=fig.get_facecolor())



In [ ]:

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot([x[0] for x in cost_per_ride],[y[1] for y in cost_per_ride],
         'r^',[x[0] for x in cost_per_ride],[y[1] for y in cost_per_ride],'r--',
        linewidth=.7)
ax2.plot([x[0] for x in cost_per_ride[:-1]],[y for y in deriv_cpr],
         'bs',[x[0] for x in cost_per_ride[:-1]],[y for y in deriv_cpr],'b--',
        linewidth=.7)

fmt = '${x:,.2f}'
tick = mtick.StrMethodFormatter(fmt)
plt.title('Economic Advantage of Mobility Services as a Function of Fare Price',size=14)
ax1.xaxis.set_major_formatter(tick) 

ax.set_ylim(ymin=-.05)
ax.set_xlim(xmin=-.05)

#Trying to format y axis
#ax1.gca().yaxis.set_major_formatter(PercentFormatter(xmax=1))
ax1.grid()
ax1.set_ylabel('% of Population Better Off Using Mobility Services (%)',size=9)
ax1.set_xlabel('Fare Per Mile ($/mile)',size=12)
ax1.yaxis.set_major_formatter(PercentFormatter(xmax=1))
ax1.tick_params('y', colors='r')

ax2.set_ylabel('Slope of Advantage Curve (%-mile/$)',size=12,color='k')
ax2.tick_params('y', colors='b')

plt.tight_layout()
fig=plt.gcf()
fig.savefig('functionoffareprice.png',dpi=600,facecolor=fig.get_facecolor())



In [ ]:
plt.plot([x[0] for x in cost_per_ride[:-1]],[y for y in deriv_cpr],
         'r^',[x[0] for x in cost_per_ride[:-1]],[y for y in deriv_cpr],'k--',
        linewidth=.7)

# print(len([x[0] for x in cost_per_ride[:-1]]))
# print(len([y for y in deriv_cpr]))

fmtx = '${x:,.2f}'
fmty = '{x:,.2f}'
tickx = mtick.StrMethodFormatter(fmtx)
# ticky = mtick.FormatStrFormatter(fmty)
ax=plt.gca()
plt.title('Derivative of Adoption of Mobility Services\nas a Function of Fare Price',size=15)

ax.xaxis.set_major_formatter(tickx) 
# ax.yaxis.set_major_formatter(ticky)

#Trying to format y axis
# plt.gca().yaxis.set_major_formatter(PercentFormatter(xmax=1))
ax.grid()
plt.ylabel('Slope of Adoption Rate(%-mile/$)',size=10)
plt.xlabel('Fare Per Mile ($/mile)',size=10)
fig=plt.gcf()
# ax.set_ylim(ymin=-.05)
# ax.set_xlim(xmin=-.05)
fig.savefig('deriv_functionoffareprice.png',dpi=500,facecolor=fig.get_facecolor())


In [ ]:
###########################################################
#Producing Cost Per Ride to Percentage Plot
#More granular


cost_per_mile_2=np.arange(.01,.31,.01)
cost_per_ride_2=[]

for x in range(0,len(cost_per_mile_2)):

    #Monte Carlo Simulation
    output_year = (7,)
    Decision = np.zeros((1,simsize))
    Car_NPV = []
    Uber_NPV = []

    for i in range(0,simsize-1):
    #     print 'Time Worth', Time_Worth[i]
    #     print 'Annual_Miles_Avg Worth', Annual_Miles_Avg[i]
    #     print 'Car Price', Car_Price[i]

        a,b,c = Car_Ownsership_Expense_Model(outputyears=output_year,annualmiles = Annual_Miles_Avg[i],carprice=Car_Price[i],timeworth= Time_Worth[i], triptime= Trip_Time_Avg[i])
        d = Uber_Expense_Model(farepermile=cost_per_mile_2[x],outputyears= output_year,timeworth=Time_Worth[i],annualmiles=Annual_Miles_Avg[i],carprice = Car_Price[i],triptime= Trip_Time_Avg[i])
        Car_NPV.append(a)
        Uber_NPV.append(d)

    #     print 'Car Own Cost', a[0]
    #     print 'Uber Cost', d
        #d is expense of ubering
        Decision[0,i] = a[0]-d



    Decision = np.transpose(Decision)
    n_bins=100
    Decision=Decision/1000 


    Uber_wins = 0 
    Car_wins = 0
    for i in range(0,len(Decision)):
        if Decision[i] < 0:
            Car_wins += 1
        else:
            Uber_wins += 1

    Uber_percent = float(Uber_wins)/len(Decision)
    Car_percent = float(Car_wins)/len(Decision)
    
    print("Cost Per Mile is ", cost_per_mile[x])
    print(Uber_percent, 'of the US is better off Ubering')
    print(Car_percent, 'of the US is better off Car Owning')

    cost_per_ride_2.append((cost_per_mile_2[x],Uber_percent))


In [ ]:
plt.style.use('seaborn-white')
# plt.style.use('dark_background')
# plt.style.use('fivethirtyeight')

plt.plot([x[0] for x in cost_per_ride_2],[y[1] for y in cost_per_ride_2],
         'r^',[x[0] for x in cost_per_ride_2],[y[1] for y in cost_per_ride_2],'k--',
        linewidth=.7)

fmt = '${x:,.2f}'
tick = mtick.StrMethodFormatter(fmt)
ax=plt.gca()
plt.title('Adoption of Mobility Services as a Function of Fare Price',size=15)
ax.xaxis.set_major_formatter(tick) 

#Trying to format y axis
plt.gca().yaxis.set_major_formatter(PercentFormatter(xmax=1))

plt.ylabel('% of Population Better Off Using Mobility Services (%)',size=10)
plt.xlabel('Fare Per Mile ($/mile)',size=10)

ax.grid()
fig=plt.gcf()
fig.savefig('functionoffareprice_zoom.png',dpi=500,bbox_inches='tight',facecolor=fig.get_facecolor())